In [11]:
%pip install pandas  -q
%pip install scikit-learn -q
%pip install matplotlib -q
%pip install seaborn -q
%pip install numpy -q
%pip install scipy -q
%pip install ucimlrepo -q 
%pip install dataclasses -q



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [30]:
import sys
import os

# Get the path to the project root directory (PBGSK)
# 'os.path.abspath('..')' goes one level up from the 'notebooks' directory
project_root = os.path.abspath('..')
print(f"Project root directory: {project_root}")
# Add the project root to the system path if it's not already there
if project_root not in sys.path:
    sys.path.append(project_root)

# Now you can import from 'src' as if it's a top-level package
from src.utils.data_importer import data_loader

# --- Example ---
# If your data_importer.py has a function like:
# def load_data(path):
#     ...
#
# You would import it like this:
# from src.utils.data_importer import load_data

Project root directory: c:\Users\romul\Repositories\pBGSK


ImportError: cannot import name 'data_loader' from 'src.utils.data_importer' (c:\Users\romul\Repositories\pBGSK\src\utils\data_importer.py)

In [ ]:
import pandas as pd
#from ucimlrepo import fetch_dataset
import random
import numpy as np
import sys


In [29]:
data_loader('tic_tac_toe_endgame')

NameError: name 'data_loader' is not defined

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
#from ucimlrepo import fetch_dataset

In [ ]:
class Data:
    def __init__(self,dataset_name):
        dataset = data_loader(dataset_name)
        self.features = dataset.data.features
        self.targets = dataset.data.targets
        self.total_features = self.features.shape[1]
        self.X_train, self.X_test, self.y_train, self.y_test = sk.model_selection.train_test_split(self.features, self.targets, test_size=0.33, random_state=42)

        

In [ ]:

class Individual:
    def __init__(self,data_set_name: str,individual_id :int, number_of_features: int, features: list):
        """Cada individuo possui um vetor binário representando a seleção de features."""
        self.data_set_name = data_set_name
        self.number_of_features = number_of_features
        self.individual_id = individual_id
        self.features = features
        self.score = None
    def set_score(self, score: float):
        self.score = score
class Population:
    def __init__(self, individuals: list):
        self.individuals = individuals
        

In [ ]:
def population_creation(num_population:int, data_set_name:str, lower_k :int, upper_k:int,dataset_id:int,total_features:int):
    """Criando uma população inicial de indivíduos. com features variando entre lower_k e upper_k.
    num_population: número de indivíduos na população"""
    population = []
    for i in range(num_population):
        k = lower_k + random.random()*(upper_k-lower_k)
        k = int(k)
        features = random.sample(range(0, total_features), k)
        bin_feature = np.zeros(total_features)
        for feature in features:
            bin_feature[feature] = 1  # Mark selected
        
        indiv = Individual(data_set_name =data_set_name,dataset_id = dataset_id ,individual_id= i,number_of_features= k,features= bin_feature)
        population.append(indiv)
    pop =Population(population)
    return pop

In [16]:
#def knn_score(indiv: individual):
    """Calcula a acurácia do classificador KNN para o indivíduo fornecido."""
    
    #return accuracy

IndentationError: unexpected indent (292593322.py, line 2)

In [17]:
def stage_dimension(gem_max:int, gem_atual:int,k_rate:float, num_individuos:int):
    """gem_max é numero máximo de gerações
        gem_atual é a geração atual
        d_junior é o número de indivíduos juniores
        d_senior é o número de indivíduos seniores
        k_rate é o knowledge rate"""
        
    d_junior = int(num_individuos*((gem_max - gem_atual)/gem_max)**k_rate)
    d_senior = num_individuos - d_junior
    return d_junior, d_senior


In [ ]:
pop =population_creation(num_population =15,data_set_name = 'tic', lower_k=3, upper_k=6, total_features=9)
#for indiv in pop:
#    print(indiv.number_of_features, indiv.features)
    

In [21]:
pop[0].features

array([0., 1., 1., 1., 1., 0., 0., 1., 0.])